## Day 1
The puzzle for day 1 can be found [here](https://adventofcode.com/2025/day/1). In essence, we are tasked with counting the number of times a safe dial stops in the very middle given a set of instructions.

As usual, we will solve the sample input first before moving onto the real problem. Let's load the sample input first.

In [1]:
with open('sample_input.txt', 'r') as f:
    lines = f.read().splitlines()

print(lines)

['L68', 'L30', 'R48', 'L5', 'R60', 'L55', 'L1', 'L99', 'R14', 'L82']


## Key insight
The rotation over the middlepoint of the dial can be considered as a modulo 100 operation:
1. Rotating towards the left can be seen as a subtraction
2. Rotation towards the right as addition
3. The starting position of the dial is 50

This can then be solved in a loop sequentially following the instructions and counting the number of times we reach exactly zero.

In [2]:
def instructions_parser(s: str) -> int:
    number = int(s[1:])
    if s[0] == 'R':
        return number
    else:
        return -number

start = 50
count = 0
for instruction in lines:
    start = (start + instructions_parser(instruction)) % 100
    if start == 0:
        count += 1

print(count)

3


## Solving the real puzzle
Now we simply load the lines from the problem and solve!

In [3]:
with open('puzzle_input.txt', 'r') as f:
    lines = f.read().splitlines()

start = 50
count = 0
for instruction in lines:
    start = (start + instructions_parser(instruction)) % 100
    if start == 0:
        count += 1

print(count)

1040


## Bonus question

For the bonus question, we are instead instructed to count the number of times the dial crosses zero instead of solely when it lands on it.

>Note: some instructions can be larger than 100 i.e. 'L595', meaning you can end up crossing zero more than once per instruction.

To solve this bonus problem, we will adapt our approach in two ways:
1. Edit our instructions parser function to also return the number of time it crosses zero per instruction
2. Edit the count handling in the for loop to also count when it rotates over zero.

The first can be done via integer division by realising that with instructions like 'L595' you are crossing zero 595 // 100 = 5 times. This will be hereforth termed as zeroes.

The second can be done by checking:
- if instruction is positive: `start + instruction % 100` > 99
- elif instruction is negative: `start != 0 and start + instruction + zeroes * 100` ≤ 0. This checks if (has to be done this way due to how modulo deals with negatives)

To explain these two cases, the first checks if we've had to cross zero without double counting i.e instructions bigger than 100. The second checks essentially the same thing but for the negative case, the modulo operator is replaced by the remainder subtracting the number of zeroes (due to how modulo works with negative numbers) and takes into account cases where you start at 0 and rotate left where you're not rotating over zero any extra times.

In [4]:
def instructions_parser2(s: str) -> tuple[int, int]:
    number = int(s[1:])
    zeroes = abs(number // 100)
    if s[0] == 'R':
        return number, zeroes
    else:
        return -number, zeroes

start = 50
count = 0

for instruction in lines:
    number, zeroes = instructions_parser2(instruction)
    # Case number is positive
    if number > 0 and start + number % 100 > 99:
        count += 1

    elif number < 0 and start != 0 and start + (number + zeroes * 100) <= 0:
        count += 1
        
    start = (start + number) % 100
    count += zeroes

print(count)


6027
